In [1]:
import os
import plotly.express as px
import pandas as pd
import numpy as np
import json

from utils import *
from train_eval import *
import wandb
wandb.login()

import warnings
warnings.filterwarnings('ignore')

# Set seed
np.random.seed(42)

# Set working directory
os.chdir(r"..") # should be the git repo root directory, checking below:
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast"
dir_path = os.path.join(os.getcwd(), 'data', 'clean_data')
model_dir = os.path.join(os.getcwd(), 'models')



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


Current working directory: c:\Users\nik\Desktop\Berkeley_Projects\WattCast


In [2]:
def train_eval_light():

    wandb.init(project="WattCast_tuning")
    wandb.config.update(config_run)
    config = wandb.config

    print("Getting data...")

    pipeline, ts_train_piped, ts_val_piped, ts_test_piped, ts_train_weather_piped, ts_val_weather_piped, ts_test_weather_piped, trg_train_inversed, trg_val_inversed, trg_test_inversed = data_pipeline(config)

    print("Getting model instance...")
    model = get_model(config)
    model, runtime = train_models([model], ts_train_piped, ts_train_weather_piped, ts_val_piped, ts_val_weather_piped)

    print("Evaluating model...")
    predictions, score = predict_testset(model[0], 
                                  ts_test_piped[config.longest_ts_test_idx], 
                                  ts_test_weather_piped[config.longest_ts_test_idx],
                                  config.n_lags, config.n_ahead, config.eval_stride, pipeline,
                                  )


    print("Plotting predictions...")
    df_compare = pd.concat([trg_test_inversed.pd_dataframe(), predictions], axis=1).dropna()
    df_compare.columns = ['target', 'prediction']
    fig = px.line(df_compare, title='Predictions vs. Test Set')

    wandb.log({'eval_loss': score})
    wandb.log({'predictions': fig})
    wandb.finish()



In [3]:
# See what keys are in the h5py data file
get_hdf_keys(dir_path)

({'1_county.h5': ['Los_Angeles', 'New_York', 'Sacramento'],
  '2_town.h5': ['town_0', 'town_1', 'town_2'],
  '3_village.h5': [],
  '4_neighborhood.h5': ['germany'],
  '5_household.h5': ['household_0', 'household_1', 'household_2'],
  '6_apartment.h5': ['apartment_0', 'apartment_1', 'apartment_2']},
 {'1_county.h5': ['60min'],
  '2_town.h5': ['15min', '60min'],
  '4_neighborhood.h5': ['15min', '60min'],
  '5_household.h5': ['15min', '60min'],
  '6_apartment.h5': ['15min', '5min', '60min']})

In [4]:
# run parameters

sweeps = 20

scale_location_pairs = (
    # ('1_county', 'Sacramento'),
    # ('1_county', 'New_York'),
    # ('2_town', 'town_1'),
    # ('2_town', 'town_2'),
    ('3_village', 'village_1'),
    ('3_village', 'village_2'),
    #('2_town', 'town_0'),
    #('3_village', 'village_0'),
    #('4_neighborhood', 'germany'),
    #('5_household', 'household_0'),
      )




models = [
        'rf',
        'xgb', 
        'gru', 
        'lgbm',  
        'nbeats',
        #'transformer',
        #'tft'
        ]

for scale, location in scale_location_pairs:

    for model in models:
        # place holder initialization of config file (will be updated in train_eval_light()
        config_run = {
            'spatial_scale': scale,
            'temp_resolution': 60,
            'location': location,
            'model': model,
            'horizon_in_hours': 24,
            'lookback_in_hours': 24,
            'boxcox': True,
            'liklihood': None,
            'weather': True,
            'holiday': True,
            'datetime_encodings': False,
        }

        with open(f'sweep_configurations/config_sweep_{model}.json', 'r') as fp:
            sweep_config = json.load(fp)                  

        sweep_config['name'] = model + 'sweep' + config_run['spatial_scale'] + '_' + config_run['location'] + '_' + str(config_run['temp_resolution'])

        sweep_id = wandb.sweep(sweep_config, project="WattCast_tuning")
        wandb.agent(sweep_id, train_eval_light, count=sweeps)


Create sweep with ID: yuvw2w3x
Sweep URL: https://wandb.ai/wattcast/Wattcast_tuning/sweeps/yuvw2w3x


wandb: Agent Starting Run: td3bykr6 with config:
wandb: 	datetime_encodings: 1
wandb: 	max_depth: 10
wandb: 	min_samples_leaf: 1
wandb: 	min_samples_split: 2
wandb: 	n_estimators: 200
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'datetime_encodings' was locked by 'sweep' (ignored update).


Getting data...


Run td3bykr6 errored: KeyError('No object named village_1/60min/train_target in the file')
wandb: ERROR Run td3bykr6 errored: KeyError('No object named village_1/60min/train_target in the file')
wandb: Agent Starting Run: 1mgz8865 with config:
wandb: 	datetime_encodings: 0
wandb: 	max_depth: 2
wandb: 	min_samples_leaf: 5
wandb: 	min_samples_split: 5
wandb: 	n_estimators: 500
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

Getting data...


wandb: Ctrl + C detected. Stopping sweep.


Create sweep with ID: nmsmq4jn
Sweep URL: https://wandb.ai/wattcast/Wattcast_tuning/sweeps/nmsmq4jn


wandb: Agent Starting Run: crx1oeeh with config:
wandb: 	batch_size: 32
wandb: 	datetime_encodings: 0
wandb: 	dropout: 0
wandb: 	hidden_dim: 128
wandb: 	lr: 0.1
wandb: 	n_rnn_layers: 1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: WARNING Ignored wandb.init() arg project when running a sweep.
